# 程式執行環境設定
1. 引用selenium套件
    - selenium docs - https://selenium-python.readthedocs.io/locating-elements.html
2.安裝webdriver，並將exe檔和code放置於同一個目錄中。注意，**伺服器版本要和webdriver執行檔版本一致(例如122.XX.XX)**
    - webdriver下載處 - https://googlechromelabs.github.io/chrome-for-testing/
3.記得在程式裡給定帳號、密碼、要爬取的帳號(twitter_list) 


In [20]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from getpass import getpass


import pandas as pd 
import time 
import csv


### 爬取X資料時常見問題
- 網頁更新之後，程式抓不到輸入框的位置，發生錯誤:
    - 修改程式碼，透過full xpath去定位網頁中的某元素。(# F12>>選取想要抓的區域>>在開發工具處code右鍵>>copy xpath)
- 輸入帳號密碼失敗:
    - 建議等畫面都跑出來再讓爬重按按鈕，可以設定sleep time，避免程式卡住
- 被X視為不明登入行為，登入失敗:
    - 同一帳號登入太頻繁，會被鎖帳，可以多辦幾隻帳號輪流登入

## 請提供帳號密碼資訊


In [16]:
my_account_emails = ['futaba2000523@gmail.com','c311706017.mg11@nycu.edu.tw', 'intheclocktown@gmail.com' ]  #帳號/email
my_usernames = ['futaba07542364','c311706017','CharinLinn'] # 暱稱  
my_passwords = ['tUtp89Cr4','tUtp89Cr4','tUtp89Cr4']  #密碼

## 請提供想要爬取的帳號列表

In [1]:
twitter_list = ['MbiyuEliud', 'koeche646', 'anericod']



'''
file_path = 'dataset\\friend\\Don1fx_bg_follower.csv'  
df = pd.read_csv(file_path)
                    
if df.empty:
    print(f"檔案 {file_path} 為空，無法讀取 twitter_list")
else:                    
    twitter_list = df.iloc[:, 0].dropna().tolist()
    twitter_list = [item.lstrip('@') for item in twitter_list]

    #twitter_lists.update({userName:twitter_list})
                    
    print("成功讀取 twitter_list:", twitter_list)
'''

## 請提供dataset的儲存位置

In [6]:
import tkinter as tk
from tkinter import filedialog

# 創建主窗口
root = tk.Tk()
root.withdraw()  # 隱藏主窗口

save_dirPath = ''

while save_dirPath=='':
# 跳出對話視窗，讓用戶選擇保存路徑
    save_dirPath = filedialog.askdirectory(title="選擇dataset的儲存位置")

print(f"dataset將保存至這裡：{save_dirPath}")


dataset將保存至這裡：C:/Users/Tina_New/Downloads/chromedriver-win64


## 爬該帳號發表的貼文
 
- 會爬取這些資料:['username','handle','postDate','textresponding','reply','retweets','likes']

In [7]:
def get_tweet_data(card):
    # # username , getting the first span tag after the current tag
    username = card.find_element(By.XPATH,'.//span').text 

    handle = card.find_element(By.XPATH,'.//span[contains(text(),"@")]').text 

    try:
        postDate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
    # Handle the missing element, e.g., set a default value or skip this data point
        postDate = None
        return
    # content of tweet 
    responding = card.find_element(By.XPATH, './/div[2]/div[2]/div[2]').text
    # print("Responding: " + responding + "\n")
    
    # reply
    if card.find_element(By.CSS_SELECTOR,'div[data-testid="reply"]').text is not None:
        reply = card.find_element(By.CSS_SELECTOR,'div[data-testid="reply"]').text
        # print("Reply " + reply + "\n")
    else:
        reply = 'None'
        # print("Reply: 0\n")
    
    # retweet 
    if card.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text is not None:
        retweets = card.find_element(By.CSS_SELECTOR,'div[data-testid="retweet"]').text
        # print("Retweets: " + retweets + "\n")
    else: 
        retweets = 'None'
        # print("Retweets: 0\n")
    
    # likes 
    if card.find_element(By.CSS_SELECTOR,'div[data-testid="like"]').text is not None:
        likes = card.find_element(By.CSS_SELECTOR,'div[data-testid="like"]').text
        # print("Likes: " + likes + "\n")
    else:
        likes = 'None'
        # print("Likes: 0\n")
    
    tweet = (username,handle,postDate,responding,reply,retweets,likes)
    return tweet

## 爬取創作者的簡介
- 會爬取這些資料:['intro','following','follower']  


In [33]:
def get_creator_data(card):
    # # username , getting the first span tag after the current tag
    username = card.find_element(By.XPATH,'.//span').text 
    # print("Username: " + username + "\n")
    # twitter handle 

    creator_page = WebDriverWait(chrome_driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div[1]/div/div/article/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[1]/div/a')))
    #/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div[1]/div/div/article/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div[1]/div/a
    chrome_driver.execute_script("arguments[0].click();", creator_page)
    
    if WebDriverWait(chrome_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//article[@data-testid="tweet"]'))):
       
     # Get the introduction, number of following, number of follower of this user
        intro_element = WebDriverWait(chrome_driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div/div/div[3]/div/div/span')))
        following_element = WebDriverWait(chrome_driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div/div/div[5]/div[1]/a/span[1]/span')))
        follower_element = WebDriverWait(chrome_driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div/div/div[5]/div[2]/a/span[1]/span')))
        
        intro = intro_element.text 
        following = following_element.text 
        follower = follower_element.text 
        
        print(intro,following,follower)
    user_bg = (intro,following,follower)
    return user_bg


0 sunnydecree
here
#Bitcoin 845 88K
1 BoredElonMusk
here
Fortune favors the BORED  4,316 1.6M
2 TuttoTechNet
here
Il nuovo punto di riferimento per la tecnologia    26 1,403
3 paul_zoell68952
here
Patriot, God, Family, Country Texas Strong,  8,808 8,739
4 schizarella
here
investigative journalist;
former  106 32.2K
5 Dany_Crypto1
here
Web3 Content Creator | representing   2,761 33.6K
6 niji_dokkta
here
독타/ドックタ/ 89 2,586
7 chichirin0603
here
左撇子畫圖仔//NOCTYX 80 1,279
8 BRcu__1996
9 joshuadecter
here
Writer, Curator, Art Historian 4,999 2,788
10 kiwi_znn
here
Fann acc for my babies  139 12.6K
11 Sance80662500
here
 EN（苦手）/中文
可能會有翻譯錯誤地方請見諒QQ
未經授權請勿使用圖與無料印製
委託可私訊詢問 487 5,832
12 itmorph1
here
animation is cool, onepiece fan 106 4,515
13 KatieePCrypto
here
PulseChain Girl attracted to real DeFi  406 41K
14 vicktop55
15 mjkmwntr
16 Drager4434
here
#crypto 151 14.2K
17 CriptoFeminino
here
₿ Bitcoin e Criptomoedas em português

Substack:  1,143 1,363
18 95pjmbz
here
jimin i eu escutando equal sig

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF67A41F922+3045234]
	(No symbol) [0x00007FF67A0C3652]
	(No symbol) [0x00007FF679F87564]
	(No symbol) [0x00007FF679FD5F9D]
	(No symbol) [0x00007FF679FD606C]
	(No symbol) [0x00007FF67A01BDD7]
	(No symbol) [0x00007FF679FF9DEF]
	(No symbol) [0x00007FF67A0191C4]
	(No symbol) [0x00007FF679FF9B53]
	(No symbol) [0x00007FF679FC813C]
	(No symbol) [0x00007FF679FC8D71]
	GetHandleVerifier [0x00007FF67A44E89D+3237613]
	GetHandleVerifier [0x00007FF67A490BA7+3508727]
	GetHandleVerifier [0x00007FF67A48976F+3478975]
	GetHandleVerifier [0x00007FF67A1FF1C6+815126]
	(No symbol) [0x00007FF67A0CED0F]
	(No symbol) [0x00007FF67A0C9854]
	(No symbol) [0x00007FF67A0C99E2]
	(No symbol) [0x00007FF67A0BA574]
	BaseThreadInitThunk [0x00007FF909EC7344+20]
	RtlUserThreadStart [0x00007FF90B0A26B1+33]


# main

In [10]:
def selenuim(my_usernames, my_account_emails, my_passwords, twitter_list, 
             short_sleep_time, long_sleep_time, desired_data_amount, save_dirPath, fileName, fileHeader):

    for i, twitter in enumerate(twitter_list[0:]):
        print(i, twitter)
    
        # changing account
        index = i%(len(my_usernames))
        my_username=my_usernames[index]
        my_account_email=my_account_emails[index]
        my_password=my_passwords[index]
    
        if (i!=0) and (i % 5 == 0):
            time.sleep(long_sleep_time)
        else:
            time.sleep(short_sleep_time)

        chrome_options = Options()
        chrome_options.add_argument('--log-level=3')
        chrome_options.add_experimental_option("detach", True)
    
        # setting the options & going to website 
        chrome_driver = webdriver.Chrome(options=chrome_options)
        # open the browser and maximize the window
        chrome_driver.maximize_window()

        chrome_driver.get('https://twitter.com/i/flow/login')
        # chrome_driver.get('https://www.reddit.com/login/')
    


        # some issue with twitter's flow login page - https://stackoverflow.com/questions/73748693/twitter-logging-automatically-by-using-the-selenium-module-unable-to-locate-ele
        username = WebDriverWait(chrome_driver, 40).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '[name="text"]'))
        )
        time.sleep(3) # attempt to scroll again 

        username.send_keys(my_account_email) # Email

        ## find element using xpath /html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]/div
        login_button = chrome_driver.find_element(By.XPATH,'/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]/div')
        chrome_driver.execute_script("arguments[0].click();", login_button)

        # verification of username 
        verify_username = WebDriverWait(chrome_driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '[name="text"]'))
        )
        verify_username.send_keys(my_username) # @username
        verify_login_button = chrome_driver.find_element(By.XPATH,'/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/button/div')
                                                         #'/html/body/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/div')

        # Wait for 10 seconds before clicking the login button
        chrome_driver.execute_script("arguments[0].click();", verify_login_button)

       
        time.sleep(10)

        my_password_input = WebDriverWait(chrome_driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '[name="password"]'))
        )
        my_password_input.send_keys(my_password)
        time.sleep(3) # attempt to scroll again 

        # log in button second
        login_button_second = chrome_driver.find_element(By.XPATH,'/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/button/div')
                                                         #'/html/body/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div')
        chrome_driver.execute_script("arguments[0].click();", login_button_second)


        search_input = WebDriverWait(chrome_driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/div/div[2]/div/input'))
                                        #'/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input'))
        )
 
        # search_input = chrome_driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input')
        # search_input.send_keys(f'(@{twitter}) min_replies:5 min_faves:10 lang:en until:2023-04-01 since:2022-09-01 -filter:retweets -filter:replies')
        search_input.send_keys(f' (from:{twitter})  lang:en until:2024-02-24 since:2022-02-10 -filter:replies')#高階搜尋

        search_input.send_keys(Keys.RETURN)

        latest_tab = WebDriverWait(chrome_driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[2]/a'))   
        )                                        
        chrome_driver.execute_script("arguments[0].click();", latest_tab)


        # chrome_driver.execute_script('window_scrollTo(0, document.body.scrollHeight);')

        # help from : https://stackoverflow.com/questions/70379706/driver-find-elements-by-xpath-divdata-testid-tweet-gives-no-output
        # getting a list of cards that contains the tweet cards 
        tweet_data = []
        tweet_ids = set()
        last_position = chrome_driver.execute_script("return window.pageYOffset;")
        scrolling = True
        try:
            while scrolling:
        
                if WebDriverWait(chrome_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//article[@data-testid="tweet"]'))):
                    cards = chrome_driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
        
                if len(cards) > 0:
                    # only care about the last 15 items 
                    for card in cards:
                        data = get_tweet_data(card)
                        
                        if data:
                            tweet_id = ''.join(str(item) for item in data)
                            if tweet_id not in tweet_ids:
                                tweet_ids.add(tweet_id)
                                tweet_data.append(data)
                else:
                    # Print text in red
                    raise NoSuchElementException('No cards were found')
                scrollAttempt = 0
                while True:
                    # check scroll position
                    chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
                    time.sleep(3)
                    currentPosition = chrome_driver.execute_script("return window.pageYOffset;")
                    if last_position == currentPosition:
                        scrollAttempt += 1
                
                        # end of scroll region 
                        if scrollAttempt >= 3:
                            scrolling = False
                            break
                        else:
                            time.sleep(5) # attempt to scroll again 
                    else:
                        last_position = currentPosition
                        break
            chrome_driver.quit()
            
        # saving the tweet 
            fileName = twitter+'_'+fileName
            with open(f'{save_dirPath+fileName}.csv','w',newline='', encoding='utf-8') as f:
                #fileHeader = header
                writer = csv.writer(f)
                writer.writerow(fileHeader)
                writer.writerows(tweet_data)
        except Exception as e:
            print(f"No data: {e}")            
            chrome_driver.quit()
            continue





In [18]:
short_sleep_time = 10  
long_sleep_time = 500 
desired_data_amount = 300
fileName = '' 
fileHeader = ['username','handle','postDate','textresponding','reply','retweets','likes']

In [19]:
#selenuim_like
selenuim(my_usernames, my_account_emails, my_passwords, twitter_list, 
             short_sleep_time, long_sleep_time, desired_data_amount, save_dirPath, fileName, fileHeader)


0 MbiyuEliud
No data: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF66E71EEA2+31554]
	(No symbol) [0x00007FF66E697ED9]
	(No symbol) [0x00007FF66E55872A]
	(No symbol) [0x00007FF66E52D995]
	(No symbol) [0x00007FF66E5D44D7]
	(No symbol) [0x00007FF66E5EC051]
	(No symbol) [0x00007FF66E5CCDD3]
	(No symbol) [0x00007FF66E59A33B]
	(No symbol) [0x00007FF66E59AED1]
	GetHandleVerifier [0x00007FF66EA28B1D+3217341]
	GetHandleVerifier [0x00007FF66EA75AE3+3532675]
	GetHandleVerifier [0x00007FF66EA6B0E0+3489152]
	GetHandleVerifier [0x00007FF66E7CE776+750614]
	(No symbol) [0x00007FF66E6A375F]
	(No symbol) [0x00007FF66E69EB14]
	(No symbol) [0x00007FF66E69ECA2]
	(No symbol) [0x00007FF66E68E16F]
	BaseThreadInitThunk [0x00007FFDF9BF7344+20]
	RtlUserThreadStart [0x00007FFDFA83CC91+33]

1 koeche646


AttributeError: 'NoneType' object has no attribute 'is_displayed'